# Part 7.4: 中介分析 (Mediation Analysis)

## 学习目标

1. 理解中介效应的定义和意义
2. 掌握直接效应和间接效应的分解
3. 学习因果中介分析框架
4. 实现从零中介分析算法
5. 应用于真实业务场景

---

## 业务场景：优惠券如何提升转化？

想象你是某电商平台的数据科学家。A/B测试显示，发送优惠券可以提升15%的购买转化率。

**老板的追问**：
- 优惠券是怎么起效的？
- 是因为增加了用户访问次数？
- 还是提高了单次访问的购买意愿？
- 如果不发券，能否通过其他方式（如推送）达到同样效果？

**核心问题**：不仅要知道 "有没有效"，还要知道 "怎么起效"！

这就是 **中介分析 (Mediation Analysis)** 要解决的问题。

---

In [ ]:
# 环境准备
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

# 颜色配置
COLORS = {
    'primary': '#2D9CDB',
    'success': '#27AE60',
    'danger': '#EB5757',
    'warning': '#F2994A',
    'info': '#9B51E0'
}

print("✅ 环境准备完成！")

## Part 1: 核心概念

### 因果图

中介分析的典型因果结构：

```
T → M → Y   (间接路径: 通过中介)
T -----→ Y  (直接路径: 不经过中介)
```

**图形化表示**：
- **T**: 处理变量 (Treatment) - 是否发券
- **M**: 中介变量 (Mediator) - 访问次数
- **Y**: 结果变量 (Outcome) - 是否购买
- **T → M → Y**: 间接路径（处理通过中介影响结果）
- **T → Y**: 直接路径（处理直接影响结果，不经过中介）

### 效应分解

**总效应 (Total Effect, TE)**：
$$TE = E[Y(1) - Y(0)]$$

处理对结果的总影响（包括直接和间接）。

**直接效应 (Natural Direct Effect, NDE)**：
处理对结果的直接影响，不经过中介
$$NDE = E[Y(1, M(0)) - Y(0, M(0))]$$

**符号解释**：
- $Y(t, m)$ 表示当处理为 t、中介为 m 时的潜在结果
- $M(t)$ 表示当处理为 t 时的潜在中介值
- $Y(1, M(0))$ 表示：给予处理（T=1），但中介保持在控制组水平 M(0)

**间接效应 (Natural Indirect Effect, NIE)**：
处理通过中介对结果的影响
$$NIE = E[Y(1, M(1)) - Y(1, M(0))]$$

**直觉**：
- 如果给处理，中介从控制组水平 M(0) 变到处理组水平 M(1)，结果会如何变化？

**分解关系**：
$$TE = NDE + NIE$$

### 识别假设

1. **顺序忽略性 (Sequential Ignorability)**：
   - $\{Y(t,m), M(t)\} \perp T | X$ （给定协变量X，处理分配随机）
   - $Y(t,m) \perp M | T, X$ （给定T和X，中介分配随机）

2. **无混淆**：没有未观测的混淆因子同时影响T→M或M→Y

3. **正值性 (Positivity)**：所有协变量组合都有正概率接受处理和中介

---

## Part 2: 数据生成

In [ ]:
def generate_mediation_data(n=2000, seed=42):
    """
    生成中介分析数据
    场景：优惠券(T) → 访问次数(M) → 购买(Y)
    """
    np.random.seed(seed)
    
    # 协变量
    X1 = np.random.normal(0, 1, n)  # 用户活跃度
    X2 = np.random.normal(0, 1, n)  # 价格敏感度
    X = np.column_stack([X1, X2])
    
    # 处理分配（有混淆）
    propensity = 1 / (1 + np.exp(-(0.5 + 0.3*X1 + 0.2*X2)))
    T = np.random.binomial(1, propensity)
    
    # 中介变量：访问次数
    # M = f(T, X) + noise
    M = (
        2 +                    # 基线
        0.5 * T +             # 发券增加访问（T → M 系数）
        0.3 * X1 +            # 活跃度影响
        np.random.normal(0, 0.5, n)
    )
    M = np.maximum(0, M)  # 保证非负
    
    # 结果变量：购买概率（logistic模型）
    # Y = f(T, M, X)
    logit_y = (
        -2 +                   # 基线
        0.3 * T +             # 券的直接效应（T → Y 系数）
        0.5 * M +             # 访问次数效应（M → Y 系数）
        0.2 * X2 +            # 价格敏感度
        np.random.normal(0, 0.3, n)
    )
    prob_y = 1 / (1 + np.exp(-logit_y))
    Y = np.random.binomial(1, prob_y)
    
    # 真实效应（在logit scale上的近似）
    # 注意：由于Y是通过logistic变换的，实际效应是非线性的
    # 这里给出的是DGP中的系数，仅作为参考
    # - T → M 系数: 0.5
    # - M → Y 系数: 0.5 (logit scale)
    # - T → Y 系数: 0.3 (logit scale)
    # 间接效应 ≈ 0.5 * 0.5 = 0.25 (logit scale, 线性近似)
    # 实际效应需要通过模拟或估计得到
    
    return {
        'X': X,
        'T': T,
        'M': M,
        'Y': Y,
        'true_effects': {
            'direct': 0.3,      # DGP中T→Y的系数
            'indirect': 0.25,   # DGP中T→M→Y的近似效应
            'total': 0.55       # 总效应（logit scale近似）
        }
    }

# 生成数据
data = generate_mediation_data()
X, T, M, Y = data['X'], data['T'], data['M'], data['Y']

print(f"数据维度: n={len(T)}")
print(f"处理组比例: {T.mean():.2%}")
print(f"平均访问次数: {M.mean():.2f}")
print(f"购买率: {Y.mean():.2%}")
print(f"\n真实效应（DGP系数，logit scale）:")
for k, v in data['true_effects'].items():
    print(f"  {k}: {v:.3f}")

## Part 3: 探索性分析

In [ ]:
# 可视化因果路径
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=('T对M的影响', 'M对Y的影响', 'T对Y的总效应')
)

# T → M
m_t1 = M[T==1]
m_t0 = M[T==0]
fig.add_trace(go.Box(y=m_t1, name='发券', marker_color=COLORS['success']), row=1, col=1)
fig.add_trace(go.Box(y=m_t0, name='不发券', marker_color=COLORS['danger']), row=1, col=1)

# M → Y (分层)
m_bins = pd.qcut(M, q=5, duplicates='drop')
y_by_m = pd.DataFrame({'M_bin': m_bins, 'Y': Y}).groupby('M_bin')['Y'].mean()
fig.add_trace(go.Bar(
    x=[str(b) for b in y_by_m.index],
    y=y_by_m.values,
    marker_color=COLORS['primary'],
    showlegend=False
), row=1, col=2)

# T → Y
y_rate_t1 = Y[T==1].mean()
y_rate_t0 = Y[T==0].mean()
fig.add_trace(go.Bar(
    x=['发券', '不发券'],
    y=[y_rate_t1, y_rate_t0],
    marker_color=[COLORS['success'], COLORS['danger']],
    text=[f'{y_rate_t1:.1%}', f'{y_rate_t0:.1%}'],
    textposition='outside',
    showlegend=False
), row=1, col=3)

# 添加y轴标签
fig.update_yaxes(title_text="访问次数", row=1, col=1)
fig.update_yaxes(title_text="购买率", row=1, col=2)
fig.update_yaxes(title_text="购买率", row=1, col=3)

fig.update_layout(height=400, template='plotly_white', showlegend=False)
fig.show()

print(f"\n观察:")
print(f"1. 发券组访问次数更多: {m_t1.mean():.2f} vs {m_t0.mean():.2f}")
print(f"2. 访问越多，购买率越高")
print(f"3. 发券组购买率更高: {y_rate_t1:.1%} vs {y_rate_t0:.1%}")
print(f"\n问题: 发券的效应中，有多少是通过增加访问次数实现的？")

## Part 4: Baron-Kenny 方法（传统方法）

### 方法步骤

**Step 1**: 回归 Y ~ T (总效应)
$$Y = \alpha_1 + \tau \cdot T + \epsilon_1$$

**Step 2**: 回归 M ~ T (T对M的效应)
$$M = \alpha_2 + a \cdot T + \epsilon_2$$

**Step 3**: 回归 Y ~ T + M (直接效应和M的效应)
$$Y = \alpha_3 + \tau' \cdot T + b \cdot M + \epsilon_3$$

**效应分解**：
- 间接效应: $IE = a \times b$
- 直接效应: $DE = \tau'$
- 总效应: $TE = \tau = \tau' + a \times b$

### 局限性

- 假设线性关系
- 假设无交互作用
- 假设无混淆

---

In [ ]:
class BaronKennyMediation:
    """Baron-Kenny 中介分析"""
    
    def __init__(self):
        self.model_total = None
        self.model_mediator = None
        self.model_direct = None
    
    def fit(self, T, M, Y, X=None):
        """拟合三个回归模型"""
        # 准备特征
        if X is not None:
            T_X = np.column_stack([T, X])
            T_M_X = np.column_stack([T, M, X])
        else:
            T_X = T.reshape(-1, 1)
            T_M_X = np.column_stack([T, M])
        
        # Step 1: Y ~ T (+ X)
        self.model_total = LinearRegression()
        self.model_total.fit(T_X, Y)
        
        # Step 2: M ~ T (+ X)
        self.model_mediator = LinearRegression()
        self.model_mediator.fit(T_X, M)
        
        # Step 3: Y ~ T + M (+ X)
        self.model_direct = LinearRegression()
        self.model_direct.fit(T_M_X, Y)
        
        return self
    
    def get_effects(self):
        """计算效应"""
        # 系数
        tau = self.model_total.coef_[0]  # 总效应
        a = self.model_mediator.coef_[0]  # T → M
        tau_prime = self.model_direct.coef_[0]  # 直接效应
        b = self.model_direct.coef_[1]  # M → Y
        
        # 间接效应
        indirect = a * b
        
        return {
            'total': tau,
            'direct': tau_prime,
            'indirect': indirect,
            'proportion_mediated': indirect / tau if tau != 0 else 0
        }

# 应用 Baron-Kenny
bk = BaronKennyMediation()
bk.fit(T, M, Y, X)
bk_effects = bk.get_effects()

print("Baron-Kenny 中介分析结果")
print("="*60)
for key, val in bk_effects.items():
    if key == 'proportion_mediated':
        print(f"{key}: {val:.1%}")
    else:
        print(f"{key}: {val:.4f}")

print(f"\n与真实值对比:")
print(f"  直接效应: {bk_effects['direct']:.3f} vs {data['true_effects']['direct']:.3f}")
print(f"  间接效应: {bk_effects['indirect']:.3f} vs {data['true_effects']['indirect']:.3f}")

## Part 5: 数学推导

### 直接效应和间接效应的定义

#### 自然直接效应 (Natural Direct Effect, NDE)

**定义**：固定中介变量在控制组的水平，处理对结果的效应
$$NDE = E[Y(1, M(0)) - Y(0, M(0))]$$

**直觉**：如果给处理组，但保持中介在控制组水平，结果会如何变化？

#### 自然间接效应 (Natural Indirect Effect, NIE)

**定义**：固定处理在处理组，中介变化对结果的效应
$$NIE = E[Y(1, M(1)) - Y(1, M(0))]$$

**直觉**：如果给处理，中介从控制组水平变到处理组水平，结果会如何变化？

### 识别公式（Pearl推导）

在顺序忽略性假设下，可以用观测数据识别NDE和NIE。

**NDE 的识别**：
$$NDE = E_X \left[ \sum_m E[Y|T=1, M=m, X] \cdot P(M=m|T=0, X) - E[Y|T=0, M=M(0), X] \right]$$

简化为：
$$NDE = E_X \left[ \sum_m E[Y|T=1, M=m, X] \cdot P(M=m|T=0, X) \right] - E[Y|T=0, X]$$

**NIE 的识别**：
$$NIE = E_X \left[ \sum_m E[Y|T=1, M=m, X] \cdot [P(M=m|T=1, X) - P(M=m|T=0, X)] \right]$$

**关键点**：
- 外层的 $E_X[\cdot]$ 表示对协变量 X 求期望
- 内层的 $\sum_m$ 对中介变量所有可能取值求和（连续情况下是积分）
- 这些公式将反事实量（如 $Y(1, M(0))$）转化为可观测的条件期望

### 线性情况下的简化

如果满足线性关系：
- $M = \alpha_M + a \cdot T + \epsilon_M$
- $Y = \alpha_Y + \tau' \cdot T + b \cdot M + \epsilon_Y$

则：
- $NDE = \tau'$ （直接效应就是 T 对 Y 的系数）
- $NIE = a \times b$ （间接效应 = T对M的效应 × M对Y的效应）
- $TE = \tau' + a \times b$ （总效应 = 直接 + 间接）

**为什么成立？**
- T 改变 1 单位 → M 改变 a 单位 → Y 改变 a×b 单位（间接）
- T 改变 1 单位 → Y 直接改变 τ' 单位（直接）
- 总效应 = τ' + a×b

---

## Part 6: 因果中介分析（从零实现）

实现完整的因果中介框架，处理非线性关系和交互效应。

In [ ]:
class CausalMediationAnalysis:
    """
    因果中介分析
    实现 Imai, Keele, Tingley (2010) 的框架
    """
    
    def __init__(self, mediator_model=None, outcome_model=None):
        self.mediator_model = mediator_model or LinearRegression()
        self.outcome_model = outcome_model or LinearRegression()
    
    def fit(self, T, M, Y, X=None):
        """
        拟合中介模型和结果模型
        M ~ T + X
        Y ~ T + M + T*M + X
        """
        n = len(T)
        
        # 准备特征
        if X is not None:
            X_with_T = np.column_stack([T, X])
            X_with_T_M = np.column_stack([T, M, T*M, X])  # 包含交互项
        else:
            X_with_T = T.reshape(-1, 1)
            X_with_T_M = np.column_stack([T, M, T*M])
        
        # 拟合 M ~ T + X
        self.mediator_model.fit(X_with_T, M)
        
        # 拟合 Y ~ T + M + T*M + X
        self.outcome_model.fit(X_with_T_M, Y)
        
        self.T = T
        self.M = M
        self.Y = Y
        self.X = X
        
        return self
    
    def predict_mediator(self, T, X=None):
        """预测中介变量"""
        if X is not None:
            X_pred = np.column_stack([T, X])
        else:
            X_pred = T.reshape(-1, 1)
        return self.mediator_model.predict(X_pred)
    
    def predict_outcome(self, T, M, X=None):
        """预测结果变量"""
        if X is not None:
            X_pred = np.column_stack([T, M, T*M, X])
        else:
            X_pred = np.column_stack([T, M, T*M])
        return self.outcome_model.predict(X_pred)
    
    def estimate_effects(self, n_samples=None):
        """
        估计因果中介效应
        使用模拟方法（参数化g-formula）
        """
        if n_samples is None:
            n_samples = len(self.T)
        
        # 使用观测数据的协变量
        if self.X is not None:
            X_sim = self.X
        else:
            X_sim = None
        
        n = len(X_sim) if X_sim is not None else n_samples
        
        # Y(1, M(1))
        T1 = np.ones(n)
        M1 = self.predict_mediator(T1, X_sim)
        Y_1_M1 = self.predict_outcome(T1, M1, X_sim)
        
        # Y(0, M(0))
        T0 = np.zeros(n)
        M0 = self.predict_mediator(T0, X_sim)
        Y_0_M0 = self.predict_outcome(T0, M0, X_sim)
        
        # Y(1, M(0)) - NDE
        Y_1_M0 = self.predict_outcome(T1, M0, X_sim)
        
        # Y(0, M(1)) - NIE (alternative)
        Y_0_M1 = self.predict_outcome(T0, M1, X_sim)
        
        # 计算效应
        total_effect = np.mean(Y_1_M1 - Y_0_M0)
        nde = np.mean(Y_1_M0 - Y_0_M0)
        nie = np.mean(Y_1_M1 - Y_1_M0)
        
        return {
            'total': total_effect,
            'direct': nde,
            'indirect': nie,
            'proportion_mediated': nie / total_effect if total_effect != 0 else 0
        }

# 应用因果中介分析
cma = CausalMediationAnalysis()
cma.fit(T, M, Y, X)
cma_effects = cma.estimate_effects()

print("因果中介分析结果")
print("="*60)
for key, val in cma_effects.items():
    if key == 'proportion_mediated':
        print(f"{key}: {val:.1%}")
    else:
        print(f"{key}: {val:.4f}")

print(f"\n与真实值对比:")
print(f"  直接效应: {cma_effects['direct']:.3f} vs {data['true_effects']['direct']:.3f}")
print(f"  间接效应: {cma_effects['indirect']:.3f} vs {data['true_effects']['indirect']:.3f}")

## Part 7: Bootstrap 置信区间

In [ ]:
def bootstrap_mediation_ci(T, M, Y, X=None, n_bootstrap=500, alpha=0.05):
    """
    计算中介效应的Bootstrap置信区间
    
    参数:
        T, M, Y, X: 数据
        n_bootstrap: Bootstrap次数
        alpha: 显著性水平
    
    返回:
        各效应的置信区间
    """
    n = len(T)
    results = {'direct': [], 'indirect': [], 'total': []}
    
    for i in range(n_bootstrap):
        # 重采样
        idx = np.random.choice(n, size=n, replace=True)
        T_boot = T[idx]
        M_boot = M[idx]
        Y_boot = Y[idx]
        X_boot = X[idx] if X is not None else None
        
        # 拟合模型
        try:
            cma_boot = CausalMediationAnalysis()
            cma_boot.fit(T_boot, M_boot, Y_boot, X_boot)
            effects = cma_boot.estimate_effects()
            
            results['direct'].append(effects['direct'])
            results['indirect'].append(effects['indirect'])
            results['total'].append(effects['total'])
        except:
            pass
    
    # 计算置信区间
    ci = {}
    for key in results:
        vals = np.array(results[key])
        ci[key] = {
            'mean': np.mean(vals),
            'std': np.std(vals),
            'ci_lower': np.percentile(vals, 100 * alpha / 2),
            'ci_upper': np.percentile(vals, 100 * (1 - alpha / 2))
        }
    
    return ci

# 计算Bootstrap置信区间
print("计算 Bootstrap 置信区间（这可能需要一些时间）...")
boot_ci = bootstrap_mediation_ci(T, M, Y, X, n_bootstrap=200)

print("\nBootstrap 置信区间")
print("="*60)
for key, val in boot_ci.items():
    print(f"\n{key.capitalize()} Effect:")
    print(f"  均值: {val['mean']:.4f}")
    print(f"  标准误: {val['std']:.4f}")
    print(f"  95% CI: [{val['ci_lower']:.4f}, {val['ci_upper']:.4f}]")

## Part 8: 可视化效应分解

In [ ]:
# 对比不同方法的结果
methods = ['Baron-Kenny', 'Causal Mediation']
direct_effects = [bk_effects['direct'], cma_effects['direct']]
indirect_effects = [bk_effects['indirect'], cma_effects['indirect']]

fig = go.Figure()

fig.add_trace(go.Bar(
    name='直接效应',
    x=methods,
    y=direct_effects,
    marker_color=COLORS['primary']
))

fig.add_trace(go.Bar(
    name='间接效应',
    x=methods,
    y=indirect_effects,
    marker_color=COLORS['success']
))

# 添加真实值参考线
fig.add_hline(
    y=data['true_effects']['direct'],
    line_dash="dash",
    line_color=COLORS['danger'],
    annotation_text="真实直接效应",
    annotation_position="right"
)

fig.update_layout(
    title='中介效应分解：方法对比',
    xaxis_title='方法',
    yaxis_title='效应大小',
    barmode='group',
    template='plotly_white',
    height=500
)

fig.show()

# 效应占比饼图
proportion_pct = cma_effects['proportion_mediated'] * 100
fig2 = go.Figure(data=[go.Pie(
    labels=['直接效应', '间接效应'],
    values=[cma_effects['direct'], cma_effects['indirect']],
    marker_colors=[COLORS['primary'], COLORS['success']]
)])

fig2.update_layout(
    title=f"效应分解：间接效应占总效应的 {proportion_pct:.1f}%",
    template='plotly_white',
    height=400
)

fig2.show()

## 思考题与练习

### 基础理解

**1. 直接效应和间接效应的区别是什么？用生活例子解释。**

<details>
<summary>点击查看参考答案</summary>

**例子：学习时间对成绩的影响**

假设增加学习时间（T）对考试成绩（Y）有两种影响路径：
- **间接效应**：学习时间 → 知识掌握度（M）→ 成绩
  - 多学习让你掌握更多知识，从而提高成绩
- **直接效应**：学习时间 → 成绩（不经过知识掌握）
  - 例如：长时间学习让你对考试更熟悉、更自信，即使知识掌握度相同也能发挥更好

**关键区别**：
- 直接效应是"控制住中介后，处理仍有的效应"
- 间接效应是"通过改变中介，处理产生的效应"

</details>

**2. 为什么说Baron-Kenny方法有局限性？什么情况下会失效？**

<details>
<summary>点击查看参考答案</summary>

**局限性**：
1. **假设线性关系**：如果T→M或M→Y是非线性的，估计会有偏
2. **忽略交互效应**：如果T和M对Y有交互作用（T×M），无法正确分解
3. **假设无混淆**：如果存在未观测的混淆变量，估计会有偏

**失效的情况**：
- 中介或结果变量是二元的（非线性）
- 存在T和M的交互作用
- 中介和结果之间有未观测的混淆

**改进方法**：使用因果中介分析框架（如本节的CausalMediationAnalysis）

</details>

**3. 因果中介分析需要哪些识别假设？哪个最难验证？**

<details>
<summary>点击查看参考答案</summary>

**三大假设**：
1. **顺序忽略性** (Sequential Ignorability)
   - 给定X，T的分配与潜在结果独立
   - 给定T和X，M的分配与潜在结果独立
2. **无混淆**：没有未观测的混淆影响T→M或M→Y
3. **正值性**：所有X的取值下，都有正概率接受T和M

**最难验证**：**顺序忽略性的第二部分**（M的分配）
- 因为M是观测变量，不是随机分配的
- 可能存在未观测的混淆同时影响M和Y
- 例如：在优惠券例子中，用户的"购买意愿"可能同时影响访问次数和购买

**应对方法**：敏感性分析（评估未观测混淆的影响）

</details>

---

### 深入分析

**4. 如果中介变量M和结果变量Y都是二元的，应该如何修改模型？**

<details>
<summary>点击查看参考答案</summary>

**修改方案**：
1. **中介模型**：使用Logistic回归
   ```python
   from sklearn.linear_model import LogisticRegression
   mediator_model = LogisticRegression()
   ```

2. **结果模型**：使用Logistic回归
   ```python
   outcome_model = LogisticRegression()
   ```

3. **预测时注意**：
   - Logistic模型的`.predict()`给出类别（0/1）
   - 应该使用`.predict_proba()`得到概率
   - 在计算反事实时使用概率而非类别

**关键代码修改**：
```python
# 预测M的概率
M_prob = mediator_model.predict_proba(X)[:, 1]

# 预测Y的概率（而非类别）
Y_prob = outcome_model.predict_proba(X)[:, 1]
```

</details>

**5. 如果存在多个中介变量，如何分析？**

<details>
<summary>点击查看参考答案</summary>

**两种思路**：

**方案1：顺序中介分析**
- 假设中介之间有顺序关系：T → M1 → M2 → Y
- 分别分析：
  - T → M1 → Y（M1的中介效应）
  - T → M2 → Y（M2的中介效应，控制M1）

**方案2：并行中介分析**
- 假设多个中介并行：T → M1 → Y, T → M2 → Y
- 每个中介都有独立的间接效应
- 总间接效应 = 各中介的间接效应之和

**Python实现思路**：
```python
# 并行中介
indirect_via_M1 = effect_T_on_M1 × effect_M1_on_Y
indirect_via_M2 = effect_T_on_M2 × effect_M2_on_Y
total_indirect = indirect_via_M1 + indirect_via_M2
```

**注意**：需要明确中介之间的因果关系（DAG）

</details>

**6. 中介效应的置信区间为什么要用Bootstrap而不是解析方法？**

<details>
<summary>点击查看参考答案</summary>

**原因**：

1. **间接效应是乘积**：IE = a × b
   - 即使a和b服从正态分布，a×b也不服从正态分布
   - 标准误的解析公式（Delta method）依赖正态假设

2. **非线性变换**：
   - 如果使用Logistic回归，涉及多重非线性变换
   - 解析方法难以推导

3. **Bootstrap的优势**：
   - 不依赖分布假设
   - 自动处理复杂的非线性关系
   - 实现简单、稳健

**Delta method 的问题**：
```python
# Delta method 估计间接效应的标准误
se_indirect ≈ sqrt(a^2 * se_b^2 + b^2 * se_a^2)
```
这个公式假设a和b独立，且近似正态，实际中往往不成立。

</details>

---

### 面试题

**题目1**：某公司测试新UI设计对用户留存的影响。发现新UI提升了15%留存率。追问：
- 如何判断这个效应是否通过「用户满意度」中介？
- 需要收集什么数据？
- 如何设计分析流程？

<details>
<summary>点击查看参考答案</summary>

**分析流程**：

**1. 明确因果图**
```
新UI(T) → 满意度(M) → 留存(Y)
新UI(T) -------→ 留存(Y)
```

**2. 需要收集的数据**：
- **处理变量T**：用户是否看到新UI（0/1）
- **中介变量M**：用户满意度（可以是评分、NPS等）
- **结果变量Y**：7天/30天留存（0/1）
- **协变量X**：
  - 用户特征：年龄、性别、注册时间
  - 行为特征：历史活跃度、付费情况
  - 设备特征：设备型号、操作系统

**3. 分析步骤**：

**Step 1**：验证中介路径存在
```python
# 检查 T → M
model_M = LinearRegression()
model_M.fit(T, M)
# 检查系数是否显著 ≠ 0

# 检查 M → Y（控制T）
model_Y = LogisticRegression()
model_Y.fit([T, M], Y)
# 检查M的系数是否显著
```

**Step 2**：估计中介效应
```python
cma = CausalMediationAnalysis()
cma.fit(T, M, Y, X)
effects = cma.estimate_effects()

print(f"直接效应: {effects['direct']}")
print(f"间接效应: {effects['indirect']}")
print(f"中介比例: {effects['proportion_mediated']:.1%}")
```

**Step 3**：Bootstrap置信区间
```python
ci = bootstrap_mediation_ci(T, M, Y, X, n_bootstrap=500)
# 检查间接效应的CI是否包含0
```

**4. 业务解释**：
- 如果间接效应显著且占比高（如>50%），说明新UI主要通过提升满意度来提高留存
- 如果直接效应占主导，说明新UI可能通过其他机制（如降低操作成本）提高留存
- 基于结果制定策略：
  - 如果主要通过满意度，可以考虑其他提升满意度的方式
  - 如果是直接效应，需要进一步研究具体机制

</details>

**题目2**：编码题 - 实现敏感性分析

<details>
<summary>点击查看题目</summary>

```python
def sensitivity_analysis_mediation(T, M, Y, X, rho_range):
    """
    中介分析的敏感性分析
    评估未观测混淆的影响
    
    参数:
        T, M, Y, X: 数据
        rho_range: 未观测混淆的相关系数范围（如 np.linspace(-0.5, 0.5, 11)）
    
    返回:
        不同rho下的效应估计
    """
    # TODO: 实现这个函数
    pass
```

</details>

<details>
<summary>点击查看参考答案</summary>

```python
def sensitivity_analysis_mediation(T, M, Y, X, rho_range):
    """
    中介分析的敏感性分析
    
    思路：
    1. 模拟未观测混淆变量U
    2. 假设U与M和Y的相关系数为rho
    3. 在不同rho下重新估计中介效应
    4. 观察效应估计如何随rho变化
    """
    results = []
    
    for rho in rho_range:
        # 生成未观测混淆U
        # 假设U是标准正态，与M和Y都相关
        n = len(T)
        U = np.random.randn(n)
        
        # 调整M和Y，使其与U相关（相关系数≈rho）
        M_adjusted = M + rho * U
        Y_adjusted = Y + rho * U
        
        # 重新估计中介效应（控制U）
        X_with_U = np.column_stack([X, U]) if X is not None else U.reshape(-1, 1)
        
        cma = CausalMediationAnalysis()
        cma.fit(T, M_adjusted, Y_adjusted, X_with_U)
        effects = cma.estimate_effects()
        
        results.append({
            'rho': rho,
            'direct': effects['direct'],
            'indirect': effects['indirect'],
            'total': effects['total']
        })
    
    return pd.DataFrame(results)

# 使用示例
rho_range = np.linspace(-0.3, 0.3, 11)
sensitivity_results = sensitivity_analysis_mediation(T, M, Y, X, rho_range)

# 可视化
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=sensitivity_results['rho'],
    y=sensitivity_results['indirect'],
    mode='lines+markers',
    name='间接效应'
))
fig.add_hline(y=0, line_dash="dash", line_color="red")
fig.update_layout(
    title='敏感性分析：间接效应随未观测混淆变化',
    xaxis_title='未观测混淆的相关系数 ρ',
    yaxis_title='间接效应估计值'
)
fig.show()
```

**解释**：
- 如果间接效应在整个rho范围内都显著（不跨越0），说明结论稳健
- 如果在某个rho值附近跨越0，说明结果对未观测混淆敏感

**注意**：这是一个简化的敏感性分析。实际应用中可能需要更复杂的方法（如Rosenbaum bounds）。

</details>

---

## 总结

### 核心要点

| 概念 | 定义 | 公式 |
|------|------|------|
| **总效应** | 处理对结果的总影响 | $TE = E[Y(1) - Y(0)]$ |
| **直接效应** | 不经过中介的效应 | $NDE = E[Y(1,M(0)) - Y(0,M(0))]$ |
| **间接效应** | 通过中介的效应 | $NIE = E[Y(1,M(1)) - Y(1,M(0))]$ |
| **中介比例** | 间接效应占总效应的比例 | $PM = NIE / TE$ |

### 方法对比

| 方法 | 优点 | 缺点 | 适用场景 |
|------|------|------|----------|
| **Baron-Kenny** | 简单直观 | 线性假设强 | 连续线性关系 |
| **因果中介分析** | 灵活，允许非线性和交互 | 需要更多假设 | 一般场景 |
| **敏感性分析** | 评估未观测混淆影响 | 计算复杂 | 高风险决策 |

### 实践建议

1. **画因果图**：明确T→M→Y的路径
2. **检查假设**：顺序忽略性最关键
3. **敏感性分析**：测试未观测混淆的影响
4. **业务解释**：将统计结果转化为可操作建议

### 延伸阅读

- **经典论文**：
  - Baron & Kenny (1986): "The Moderator-Mediator Variable Distinction"
  - Imai, Keele & Tingley (2010): "A General Approach to Causal Mediation Analysis"
  - Pearl (2001): "Direct and Indirect Effects"

- **Python工具**：
  - `mediation` package
  - `causalml` 中的中介分析模块

---

**恭喜完成中介分析的学习！**

你现在可以：
- ✅ 分解因果效应为直接和间接部分
- ✅ 理解"怎么起效"而不仅是"有没有效"
- ✅ 应用于真实业务场景做机制分析
- ✅ 实现从零中介分析算法
- ✅ 计算Bootstrap置信区间